In [74]:
import pandas as pd
import numpy as np
import plotly.express as px
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Conexion a la BD

In [10]:
def dataBase_connection():
    """This function connects with the data Warehouse and return a diccionary with the dataframes"""

    host = '127.0.0.1'  # Cambia esto por el endpoint de tu RDS
    port = 3307
    user = 'airflow'  # Cambia esto por tu usuario
    password = 'airflow'  # Cambia esto por tu contraseña
    database = 'airflow'  # Cambia esto por tu base de datos
    engine = None

    
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
    

    # Create a list of the tables in the database
    tables =['temperature_WH','humidity_WH','wind_speed_WH','pressure_WH','weather_description_WH','wind_direction_WH']
   
    dataframes = {}
    # Loop through the tables and read them into pandas dataframes
    for table in tables:
        df = pd.read_sql_table(table, engine)
        dataframes[table] = df

    # Close the connection
    engine.dispose()
    return dataframes

In [ ]:
#Get the dataframes from the database
dataframes = dataBase_connection()

In [ ]:
#List of the names of the tables in the database
dataframes.keys()

dict_keys(['temperature_WH', 'humidity_WH', 'wind_speed_WH', 'pressure_WH', 'weather_description_WH', 'wind_direction_WH'])

In [15]:
dataframes['temperature_WH'].head(500)

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem,id
0,2012-10-01 12:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,309.100000,0.00,0.00,0.00,1
1,2012-10-01 13:00:00,284.630000,282.080000,289.480000,281.800000,291.870000,291.530000,293.410000,296.600000,285.120000,...,288.220000,285.830000,287.170000,307.590000,305.470000,310.580000,304.40,304.40,303.50,2
2,2012-10-01 14:00:00,284.629041,282.083252,289.474993,281.797217,291.868186,291.533501,293.403141,296.608509,285.154558,...,288.247676,285.834650,287.186092,307.590000,304.310000,310.495769,304.40,304.40,303.50,3
3,2012-10-01 15:00:00,284.626998,282.091866,289.460618,281.789833,291.862844,291.543355,293.392177,296.631487,285.233952,...,288.326940,285.847790,287.231672,307.391513,304.281841,310.411538,304.40,304.40,303.50,4
4,2012-10-01 16:00:00,284.624955,282.100481,289.446243,281.782449,291.857503,291.553209,293.381213,296.654466,285.313345,...,288.406203,285.860929,287.277251,307.145200,304.238015,310.327308,304.40,304.40,303.50,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2012-10-22 03:00:00,279.150000,279.460000,286.420000,279.420000,291.990000,290.780000,297.270000,298.340000,290.260000,...,284.080000,282.710000,287.690000,293.710000,293.440000,298.830000,297.40,297.40,295.20,496
496,2012-10-22 04:00:00,278.660000,279.110000,286.070000,278.980000,291.970000,290.470000,296.640000,296.990000,288.640000,...,284.450000,282.180000,286.350000,293.710000,293.150000,300.150000,297.40,297.40,295.20,497
497,2012-10-22 05:00:00,278.640000,279.290000,285.670000,279.120000,292.000000,290.600000,295.450000,296.420000,287.510000,...,284.810000,281.200000,285.050000,296.480000,296.150000,298.150000,297.40,297.40,295.20,498
498,2012-10-22 06:00:00,278.580000,279.530000,285.560000,278.850000,291.840000,290.610000,294.440000,296.460000,287.480000,...,284.090000,281.550000,283.650000,298.150000,297.470000,298.680000,300.08,300.08,300.24,499


In [20]:
# Change the type of data int the columns datetime 
dataframes['temperature_WH']['datetime'] = pd.to_datetime(dataframes['temperature_WH']['datetime'])

In [21]:
dataframes['temperature_WH']['datetime'].dt.year.unique()

array([2012, 2013, 2014, 2015, 2016, 2017])

## Temperature

### Los Angeles

In [ ]:
def graphic(df,year,city):
    """This function creates a graphic with the temperature data for a specific year"""

    #Filter the dataframe to get only the data of 2023
    temperature_df= df[df['datetime'].dt.year == year].copy()

    #Get the average of the temperature by month
    temperature_df['month'] = temperature_df['datetime'].dt.month
    
    #Change the name of the month
    temperature_df['Month'] = temperature_df['month'].replace({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'})


    #Get the average of the temperature by month
    temperature_df = temperature_df.groupby(['Month']).mean().reset_index()

    # Show int ascending order
    temperature_df = temperature_df.sort_values(by='month', ascending=True)

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Bar( x = temperature_df['Month'], y = temperature_df[city], name = f'Temperature {year}', 
            marker= dict(color=temperature_df[city], colorscale = [[0,'rgb(255,200,200)'],[1,'rgb(255,0,0)']],# scale from red to white
                            colorbar=dict(title='Temp °F')))
   

    return fig


    

In [61]:
#Filter the dataframe to get only the data of 2023
temperature_2012= dataframes['temperature_WH'][dataframes['temperature_WH']['datetime'].dt.year == 2012].copy()

In [62]:
#Get the average of the temperature by month
temperature_2012['month'] = temperature_2012['datetime'].dt.month

In [63]:
#Change the name of the month
temperature_2012['Month'] = temperature_2012['month'].replace({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'})


In [64]:
#Get the average of the temperature by month
temperature_2012 = temperature_2012.groupby(['Month']).mean().reset_index()

In [66]:
# Show int ascending order
temperature_2012 = temperature_2012.sort_values(by='month', ascending=True)

In [31]:
temperature_2012 = temperature_2012.groupby(['month']).mean().reset_index()

In [65]:
temperature_2012

,Month,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,...,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem,id,month
0,December,2012-12-16 11:30:00,277.587581,278.555784,283.403330,278.315333,286.239209,286.137814,282.164206,285.024105,...,269.378626,275.696347,290.105003,289.103921,288.888400,290.444084,290.443663,287.932162,1824.5,12.0
1,November,2012-11-15 23:30:00,280.393083,281.761056,286.592826,281.012396,289.276347,289.010924,287.529389,291.783389,...,274.771326,278.143479,294.437090,293.712375,294.403604,294.944431,294.945764,293.177819,1092.5,11.0
2,October,2012-10-16 17:30:00,282.906773,284.821025,289.221588,283.941537,292.854532,292.420169,293.630074,297.128656,...,283.166507,285.588096,298.987389,297.838706,300.476790,298.306865,298.304945,297.374146,366.5,10.0


In [82]:
#creacion de un subplot de 2 filas 3 columnas para mostras las fraficas de todos los años
fig = make_subplots(rows=3, cols=2, subplot_titles=('2012', '2013', '2014', '2015', '2016', '2017'))
fig.add_trace(graphic(dataframes['temperature_WH'],2012,'Los Angeles'), row=1, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2013,'Los Angeles'), row=1, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2014,'Los Angeles'), row=2, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2015,'Los Angeles'), row=2, col=2)
fig.add_trace(graphic(dataframes['temperature_WH'],2016,'Los Angeles'), row=3, col=1)
fig.add_trace(graphic(dataframes['temperature_WH'],2017,'Los Angeles'), row=3, col=2)

fig.update_layout(
    height=800, width=1000,
    title_text="Temperatures in Los Angeles from 2012 to 2017",
    showlegend=False
)

fig.show()


In [83]:
def linear_graphic(df,city):
    """This function creat a linear grophic where show the temperature avarage for each year in a specific city"""
    #Get the average of the temperature by year
    df['year'] = df['datetime'].dt.year

    #Get the average of the temperature by year
    df = df.groupby(['year']).mean().reset_index()

    # Graphic of the temperature where the x axis are the months of a year and the y axis are the temperature in Farenheit degrees
    fig = go.Figure(data=go.Scatter(x=df['year'], y=df[city], mode='lines+markers', name=f'Temperature {city}'))
    
    return fig


In [85]:
#Get the rsult of the function linear_graphic
fig = linear_graphic(dataframes['temperature_WH'],'Los Angeles')
fig.update_layout(
    title="Temperature in Los Angeles from 2012 to 2017",
    xaxis_title="Year",
    yaxis_title="Temperature °F",
    showlegend=True
)
fig.show()